In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from textblob import TextBlob
import seaborn as sns
import itertools
import collections

import nltk
from nltk.corpus import stopwords
import re
import networkx
from textblob import TextBlob

import warnings
warnings.filterwarnings("ignore")


import warnings
warnings.filterwarnings("ignore")

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
df=pd.read_csv('/kaggle/input/us-election-2020-tweets/hashtag_joebiden.csv', lineterminator='\n')
df1=pd.read_csv('/kaggle/input/us-election-2020-tweets/hashtag_donaldtrump.csv', lineterminator='\n')
df['hastag']='joebiden'
df1['hastag']='donaldtrump'

# combine file hastag joebiden and donaldtrump
df_combined = pd.concat([df,df1]).reset_index()
del df_combined['index']


In [ ]:
# check isnull data
#df_combined.isnull().sum()


In [ ]:
#Trend Tweets
df_combined['created_at']=pd.to_datetime(df_combined['created_at'], infer_datetime_format=True,format='%Y/%m/%d',utc=False)
df_combined['weekday'] = df_combined['created_at'].apply(lambda time: time.dayofweek)
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]
df_combined['day_name']=df_combined['weekday'].apply (lambda x : days[x])
df_combined['hour']=df_combined['created_at'].dt.hour
df_combined['day']=df_combined['created_at'].dt.day
df_combined['date']=df_combined['created_at'].dt.date.astype('datetime64[ns]')
df_combined['weekday']=df_combined['weekday'].astype('str')


In [ ]:
#trend tweets per day

x=df_combined[df_combined['hastag']=='joebiden'].groupby(by='date')['tweet'].count().sort_values(ascending=False).index
y=df_combined[df_combined['hastag']=='joebiden'].groupby(by='date')['tweet'].count().sort_values(ascending=False)

x1=df_combined[df_combined['hastag']=='donaldtrump'].groupby(by='date')['tweet'].count().sort_values(ascending=False).index
y1=df_combined[df_combined['hastag']=='donaldtrump'].groupby(by='date')['tweet'].count().sort_values(ascending=False)

fig = go.Figure([go.Bar(x=x, y=y, name='Joe Biden'),
                 go.Bar(x=x1, y=y1, name='Donald Trump')])


fig.update_layout(title_text='Daily Tweets Joe Biden vs Donald Trump')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Count of Tweets')
fig.show()

# Hourly
#tweets per hour

x=df_combined[df_combined['hastag']=='joebiden'].groupby('hour').count().tweet.sort_values(ascending=False).index
y=df_combined[df_combined['hastag']=='joebiden'].groupby('hour').count().tweet.sort_values(ascending=False)

x1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('hour').count().tweet.sort_values(ascending=False).index
y1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('hour').count().tweet.sort_values(ascending=False)

fig = go.Figure([go.Bar(x=x, y=y, name='Joe Biden'),
                 go.Bar(x=x1, y=y1, name='Donald Trump')])


fig.update_layout(title_text='Hour Tweets Joe Biden vs Donald Trump')
fig.update_xaxes(title='Hour')
fig.update_yaxes(title='Count of Tweets')
fig.show()

#tweets per Weekdays

x=df_combined[df_combined['hastag']=='joebiden'].groupby('weekday')['weekday'].count().index
y=df_combined[df_combined['hastag']=='joebiden'].groupby('weekday')['tweet'].count()

x1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('weekday')['weekday'].count().index
y1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('weekday')['tweet'].count()

fig = go.Figure([go.Bar(x=x, y=y, name='Joe Biden'),
                 go.Bar(x=x1, y=y1, name='Donald Trump')])


fig.update_layout(title_text='Weekdays Tweets Joe Biden vs Donald Trump')
fig.update_xaxes(title='Weekdays')
fig.update_yaxes(title='Count of Tweets')
fig.show()


In [ ]:
# looking for continent top tweet

#Joe biden
y = df_combined[df_combined['hastag']=='joebiden'].dropna(subset=['continent']).groupby('continent')['tweet'].count()

x = df_combined[df_combined['hastag']=='joebiden'].dropna(subset=['continent']).groupby('continent')['tweet'].count().sort_values(ascending=False).index

#donal trump
y2 = df_combined[df_combined['hastag']=='donaldtrump'].dropna(subset=['continent']).groupby('continent')['tweet'].count()

x2 = df_combined[df_combined['hastag']=='donaldtrump'].dropna(subset=['continent']).groupby('continent')['tweet'].count().sort_values(ascending=False).index

fig = go.Figure([go.Bar(x=x, y=y, name='Joe Biden'),
                 go.Bar(x=x2, y=y2, name='Donald Trump')])


fig.update_layout(title_text='Top Continent Joe Biden vs Donald Trump Tweets')
fig.update_xaxes(title='Continnet')
fig.update_yaxes(title='Count of Tweets')
fig.show()

# change contry name united state of america become United States
df_combined['country']=df_combined['country'].apply (lambda x: 'United States' if x=='United States of America' else x)


# looking for top 20 country tweet
#Joe biden
y = df_combined[df_combined['hastag']=='joebiden'].dropna(subset=['country']).groupby('country')['tweet'].count().sort_values(ascending=False).head(20)
x = df_combined[df_combined['hastag']=='joebiden'].dropna(subset=['country']).groupby('country')['tweet'].count().sort_values(ascending=False).head(20).index

#donal trump
y2 = df_combined[df_combined['hastag']=='donaldtrump'].dropna(subset=['country']).groupby('country')['tweet'].count().sort_values(ascending=False).head(20)
x2 = df_combined[df_combined['hastag']=='donaldtrump'].dropna(subset=['country']).groupby('country')['tweet'].count().sort_values(ascending=False).head(20).index

fig = go.Figure([go.Bar(x=x, y=y, name='Joe Biden'),
                 go.Bar(x=x2, y=y2, name='Donald Trump')])


fig.update_layout(title_text='Top 20 Country Joe Biden vs Donald Trump Tweets')
fig.update_xaxes(title='Continents')
fig.update_yaxes(title='Count of Tweets')
fig.show()



In [ ]:
# looking for tweets filter continent
#filter Asia
continent='Asia'
df_combined1=df_combined[df_combined['continent']==continent]

# looking for top 20 country tweet
#Joe biden
y = df_combined1[df_combined1['hastag']=='joebiden'].dropna(subset=['country']).groupby(by='country')['tweet'].count()

x = df_combined1[df_combined1['hastag']=='joebiden'].dropna(subset=['country']).groupby(by='country')['tweet'].count().index

#donal trump
y2 = df_combined1[df_combined1['hastag']=='donaldtrump'].dropna(subset=['country']).groupby(by='country')['tweet'].count()

x2 = df_combined1[df_combined1['hastag']=='donaldtrump'].dropna(subset=['country']).groupby(by='country')['tweet'].count().index

fig = go.Figure([go.Bar(x=x, y=y, name='Joe Biden'),
                 go.Bar(x=x2, y=y2, name='Donald Trump')])


fig.update_layout(title_text='Country Tweets Continent Joe Biden vs Donald Trump '+continent)
fig.update_xaxes(title='Countries')
fig.update_yaxes(title='Count of Tweets')
fig.show()

In [ ]:
# looking for tweets by state
# filter country

country='United States'

df_combined1=df_combined[df_combined['country']==country]

# looking for top 20 country tweet
#Joe biden
y = df_combined1[df_combined1['hastag']=='joebiden'].dropna(subset=['state']).groupby('state')['tweet'].count()

x = df_combined1[df_combined1['hastag']=='joebiden'].dropna(subset=['state']).groupby(by='state')['tweet'].count().index

#donal trump
y2 = df_combined1[df_combined1['hastag']=='donaldtrump'].dropna(subset=['state']).groupby(by='state')['tweet'].count()

x2 = df_combined1[df_combined1['hastag']=='donaldtrump'].dropna(subset=['state']).groupby(by='state')['tweet'].count().index

fig = go.Figure([go.Bar(x=x, y=y, name='Joe Biden'),
                 go.Bar(x=x2, y=y2, name='Donald Trump')])


fig.update_layout(title_text='Top 20 Country Joe Biden vs Donald Trump Tweets Country '+country)
fig.update_xaxes(title='Countries')
fig.update_yaxes(title='Count of Tweets')
fig.show()

In [ ]:
# Likes Biden vs Trumps
#Biden
x=df_combined[df_combined['hastag']=='joebiden'].groupby('date')['likes'].count().index
y=df_combined[df_combined['hastag']=='joebiden'].groupby('date')['likes'].sum()

x1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('date')['likes'].count().index
y1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('date')['likes'].sum()
fig = go.Figure([go.Bar(x=x, y=y, name='Joe Biden'),
                 go.Bar(x=x1, y=y1, name='Donald Trump')])


fig.update_layout(title_text='Daily Likes Joe Biden vs Donald Trump')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Count of Likes')
fig.show()

# total of likes Biden vs Trump

x=df_combined[df_combined['hastag']=='joebiden'].groupby('hastag')['likes'].count().index
y=df_combined[df_combined['hastag']=='joebiden'].groupby('hastag')['likes'].sum()

x1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('hastag')['likes'].count().index
y1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('hastag')['likes'].sum()
fig = go.Figure([go.Bar(x=x, y=y, name='Joe Biden'),
                 go.Bar(x=x1, y=y1, name='Donald Trump')])


fig.update_layout(title_text='Total Likes Joe Biden vs Donald Trump')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Count of Likes')
fig.show()


In [ ]:
# Retweets Biden vs Trumps
#Biden
x=df_combined[df_combined['hastag']=='joebiden'].groupby('date')['retweet_count'].count().index
y=df_combined[df_combined['hastag']=='joebiden'].groupby('date')['retweet_count'].sum()

x1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('date')['retweet_count'].count().index
y1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('date')['retweet_count'].sum()
fig = go.Figure([go.Bar(x=x, y=y, name='Joe Biden'),
                 go.Bar(x=x1, y=y1, name='Donald Trump')])


fig.update_layout(title_text='Daily Retweets Joe Biden vs Donald Trump')
fig.update_xaxes(title='Date')
fig.update_yaxes(title='Count of Retweets')
fig.show()

# total of tweets Biden vs Trump

x=df_combined[df_combined['hastag']=='joebiden'].groupby('hastag')['tweet'].count().index
y=df_combined[df_combined['hastag']=='joebiden'].groupby('hastag')['tweet'].count()

x1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('hastag')['retweet_count'].count().index
y1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('hastag')['retweet_count'].count()
fig = go.Figure([go.Bar(x=x, y=y, name='Joe Biden'),
                 go.Bar(x=x1, y=y1, name='Donald Trump')])


fig.update_layout(title_text='Total Tweets Joe Biden vs Donald Trump')
fig.update_xaxes(title='Candidats')
fig.update_yaxes(title='Count of Tweets')
fig.show()

# total of likes Biden vs Trump

x=df_combined[df_combined['hastag']=='joebiden'].groupby('hastag')['likes'].count().index
y=df_combined[df_combined['hastag']=='joebiden'].groupby('hastag')['likes'].sum()

x1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('hastag')['likes'].count().index
y1=df_combined[df_combined['hastag']=='donaldtrump'].groupby('hastag')['likes'].sum()
fig = go.Figure([go.Bar(x=x, y=y, name='Joe Biden'),
                 go.Bar(x=x1, y=y1, name='Donald Trump')])


fig.update_layout(title_text='Total Likes Joe Biden vs Donald Trump')
fig.update_xaxes(title='Candidats')
fig.update_yaxes(title='Count of Likes')
fig.show()



# SENTIMENT ANALYSIS

In [ ]:
import nltk
from nltk.corpus import stopwords
import re
import networkx
from textblob import TextBlob

import warnings
warnings.filterwarnings("ignore")


In [ ]:
#cleaning the tweets
def clean_tweets(tweet):
 
    tweet = re.sub(r':', '', str(tweet))
    tweet = re.sub(r'‚Ä¶', '', str(tweet))
    tweet = re.sub('@[A-Za-z0–9]+', '', str(tweet))               
    tweet = re.sub('#', '', str(tweet)) # Removing '#' hash tag
    tweet = re.sub('https?:\/\/\S+', '', str(tweet)) # Removing hyperlink
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ',str(tweet))
    tweet = str(tweet).lower()
    tweet = re.sub('\[.*?\]', '', tweet)
    tweet = re.sub('https?://\S+|www\.\S+', '', tweet)
    tweet = re.sub('<.*?>+', '', tweet)
    tweet = re.sub('\n', '', tweet)
    tweet = re.sub('\w*\d\w*', '', tweet)
    tweet = re.sub(r'#','',tweet)
    tweet = re.sub(r'RT[\s]+','',tweet)
    tweet = re.sub(r'[^\w]', ' ', tweet)
    tweet = re.sub('https', '', tweet)
    tweet = re.sub('https ', '', tweet)
    tweet = re.sub('co', '', tweet)

    return tweet

In [ ]:
# biden
biden=df_combined[df_combined['hastag']=='joebiden']
trump=df_combined[df_combined['hastag']=='donaldtrump']


In [ ]:
## Sentiment Analysis JoeBiden

def getSubjectivity(tweet):
    try:
        return TextBlob(tweet).sentiment.subjectivity
    except:
        return None


# Create a function to get the polarity
def getPolarity(tweet):
    try:
        return  TextBlob(tweet).sentiment.polarity
    except:
        return None

def sentiment_calc(tweet):
    try:
        return TextBlob(tweet).sentiment
    except:
        return None

country='United States'
biden=biden[biden['country']==country]
biden['edited_tweet'] = biden['tweet'].apply(clean_tweets)
# Create two new columns 'Subjectivity' & 'Polarity'
biden['subjectivity'] = biden['edited_tweet'].apply(getSubjectivity)
biden['polarity'] = biden['edited_tweet'].apply(getPolarity)
biden['sentiment'] = biden['edited_tweet'].apply(sentiment_calc)

def getAnalysis(score):
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

biden['analysis'] = biden['polarity'].apply(getAnalysis)

In [ ]:
biden.groupby('analysis')['analysis'].count().plot.bar()
plt.title('Sentiment Analyisis Biden country: '+country)

In [ ]:
x=biden.groupby('analysis')['analysis'].count()
neutral=biden[biden['analysis']=='Neutral']['edited_tweet'].count()
positive=biden[biden['analysis']=='Positive']['edited_tweet'].count()
negative=biden[biden['analysis']=='Negative']['edited_tweet'].count()

fig = go.Figure(data=[go.Pie(labels=["positivity","negativity","neutrality"], values=[positive,negative,neutral])])
fig.update_layout(title_text='Sentiment analysis of Biden tweets')
fig.show()

In [ ]:
sentiment_df = pd.DataFrame(biden, columns=["polarity", "edited_tweet"])

fig, ax = plt.subplots(figsize=(8, 6))

# Plot histogram of the polarity values
sentiment_df.hist(bins=[-1, -0.75, -0.5, -0.25, 0.25, 0.5, 0.75, 1],
             ax=ax,
             color="blue")

plt.title("Sentiments from Tweets on Joe Biden")
plt.show()

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
def word_cloud(wd_list):
    stopwords = set(STOPWORDS)
    all_words = ' '.join([text for text in wd_list])
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        width=1600,
        height=800,
        random_state=1,
        colormap='jet',
        max_words=80,
        max_font_size=200).generate(all_words)
    plt.figure(figsize=(12, 10))
    plt.axis('off')
    plt.imshow(wordcloud, interpolation="bilinear");
word_cloud(biden['edited_tweet'][0:1000])

# Trump

In [ ]:
## Sentiment Analysis Trump

def getSubjectivity(tweet):
    try:
        return TextBlob(tweet).sentiment.subjectivity
    except:
        return None


# Create a function to get the polarity
def getPolarity(tweet):
    try:
        return  TextBlob(tweet).sentiment.polarity
    except:
        return None

def sentiment_calc(tweet):
    try:
        return TextBlob(tweet).sentiment
    except:
        return None

country='United States'
trump=trump[trump['country']==country]
trump['edited_tweet'] = trump['tweet'].apply(clean_tweets)
# Create two new columns 'Subjectivity' & 'Polarity'
trump['subjectivity'] = trump['edited_tweet'].apply(getSubjectivity)
trump['polarity'] = trump['edited_tweet'].apply(getPolarity)
trump['sentiment'] = trump['edited_tweet'].apply(sentiment_calc)

def getAnalysis(score):
    if score < 0:
      return 'Negative'
    elif score == 0:
      return 'Neutral'
    else:
      return 'Positive'

trump['analysis'] = trump['polarity'].apply(getAnalysis)
trump.groupby('analysis')['analysis'].count().plot.bar()
plt.title('Sentiment Analyisis Donald Trump country: '+country)

In [ ]:
x=trump.groupby('analysis')['analysis'].count()
neutral=trump[trump['analysis']=='Neutral']['edited_tweet'].count()
positive=trump[trump['analysis']=='Positive']['edited_tweet'].count()
negative=trump[trump['analysis']=='Negative']['edited_tweet'].count()

fig = go.Figure(data=[go.Pie(labels=["positivity","negativity","neutrality"], values=[positive,negative,neutral])])
fig.update_layout(title_text='Sentiment analysis of Trump tweets')
fig.show()


In [ ]:
sentiment_df = pd.DataFrame(trump, columns=["polarity", "edited_tweet"])

fig, ax = plt.subplots(figsize=(8, 6))

# Plot histogram of the polarity values
sentiment_df.hist(bins=[-1, -0.75, -0.5, -0.25, 0.25, 0.5, 0.75, 1],
             ax=ax,
             color="blue")

plt.title("Sentiments from Tweets on Trump")
plt.show()

> > ## WordCloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
def word_cloud(wd_list):
    stopwords = set(STOPWORDS)
    all_words = ' '.join([text for text in wd_list])
    wordcloud = WordCloud(
        background_color='white',
        stopwords=stopwords,
        width=1600,
        height=800,
        random_state=1,
        colormap='jet',
        max_words=80,
        max_font_size=200).generate(all_words)
    plt.figure(figsize=(12, 10))
    plt.axis('off')
    plt.imshow(wordcloud, interpolation="bilinear");
word_cloud(trump['edited_tweet'][0:1000])